# Подключение библиотек

In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import time
import struct
import os
from IPython.display import clear_output

# Подключимся к MetaTrader 5

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    quit()

# Вычисления медианы, верхненего и нижнего пределов

In [3]:
def take_rates(): 
    gbpusd_rates = mt5.copy_rates_from_pos("PEP", mt5.TIMEFRAME_M1, 0, 15)
    eurusd_rates = mt5.copy_rates_from_pos("KO", mt5.TIMEFRAME_M1, 0, 15)
    return gbpusd_rates, eurusd_rates

In [4]:
def calculation(rate1, rate2, lotko, lotpep):
    spread = rate1 - rate2 * lotko / lotpep # Вычисляем спред
    spread = np.array(spread)
    median = np.median(spread)
    Open_up = list()
    Open_down = list()
    Open_UP =0
    Open_DOWN = 0
    #Вычисляем верхнюю и нижнюю границу
    for i in range(0,len(spread)):
        if spread[i] > median:
            Open_up.append(spread[i])
        if spread[i] < median:
            Open_down.append(spread[i])
    Open_UP =np.median(Open_up)
    Open_DOWN = np.median(Open_down)
    return Open_UP, Open_DOWN, median

# Подготавливаем все запросы
## Нам нужно 8 запросов: 4 для открытия позиции, 4 для закрытия позиции.

In [5]:
symbol_pep = "PEP"
symbol_ko = "KO"
symbol_pep_info = mt5.symbol_info(symbol_pep)
symbol_ko_info = mt5.symbol_info(symbol_ko)
lotpep = 5.0
lotko = 13.0
deviation = 1

# #Краткое описание каждого аргумента

# "action": mt5.TRADE_ACTION_DEAL,      # Совершить сделку прямо сейчас

# "symbol": symbol_pep,                # Валюта сделки

# "volume": lot,                        # Объем который мы покупаем: У нас равен 1

# "type": mt5.ORDER_TYPE_SELL,          # Покупаем или продаём

# "deviation": deviation,               # Максимальное отклонение от заданной цены

# "magic": 123456,                      # Идентификатор эксперта. 
#                                       #Позволяет организовать аналитическую обработку торговых ордеров.    

# "comment": "Какой либо комментарий",  # Комменатрий к запросу

# "type_time": mt5.ORDER_TIME_GTC,      # Тип ордера по истечению: У нас Ордер будет находиться 
#                                       # в очереди до тех пор, пока не будет снят

# "type_filling": mt5.ORDER_FILLING_FOK,# Не будет продажи, если нет необходимого обьема 
    
    
request_pep_sell_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_pep,
"volume": lotpep, 
"type": mt5.ORDER_TYPE_SELL,
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, продали фунт",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,
    

}
request_ko_buy_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_ko,
"volume": lotko,
"type": mt5.ORDER_TYPE_BUY,
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, купили евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,      
}

request_pep_buy_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_pep,
"volume": lotpep,
"type": mt5.ORDER_TYPE_BUY, 
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, купили фунт",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,  
}

request_ko_sell_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_ko,
"volume": lotko,
"type": mt5.ORDER_TYPE_SELL, 
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, продали евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,      
}

# В запросах для закрытия позиции есть ещё один аргумент

#"position": order_pep_sell.order,                     # Указывает какую позицию мы закрывем


# Чтобы запросы закрытия позции знали, что закрыват откроем предыдущие позции
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
order_pep_buy_open = mt5.order_send(request_pep_buy_open)
order_pep_sell_open = mt5.order_send(request_pep_sell_open)
order_ko_buy_open = mt5.order_send(request_ko_buy_open)
order_ko_sell_open = mt5.order_send(request_ko_sell_open)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

request_pep_buy_close = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_pep, 
"volume": lotpep, 
"type": mt5.ORDER_TYPE_BUY,
"position": order_pep_sell_open.order,
"deviation": deviation, 
"magic": 123456, 
"comment": "Закрыли позицию, купили фунт", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}
request_ko_sell_close = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_ko,
"volume": lotko,
"type": mt5.ORDER_TYPE_SELL,
"position": order_ko_buy_open.order,
"deviation": deviation,
"magic": 123456,
"comment": "Закрыли позицию, продали евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   
}
request_pep_sell_close = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_pep, 
"volume": lotpep, 
"type": mt5.ORDER_TYPE_SELL,
"position": order_pep_buy_open.order,
"deviation": deviation, 
"magic": 123456, 
"comment": "Закрыли позицю, продали фунт", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}
request_ko_buy_close = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_ko,
"volume": lotko,
"type": mt5.ORDER_TYPE_BUY,
"position": order_ko_sell_open.order,
"deviation": deviation,
"magic": 123456,
"comment": "Закрыли позцию, купили евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   
}

# Функции для открытия и закрытия позиции

In [6]:
def request_open_up_pos(): 
    result = mt5.order_send(request_pep_sell_open) 
    result2 = mt5.order_send(request_ko_buy_open)
    print('Открыл сверху')
#     print(result)
#     print(result2)

In [7]:
def request_open_down_pos(): 
    result =mt5.order_send(request_pep_buy_open) 
    result2 = mt5.order_send(request_ko_sell_open)
    print('Открыл снизу')

In [8]:
def request_close_up_pos():
    result = mt5.order_send(request_pep_buy_close) 
    result2 = mt5.order_send(request_ko_sell_close)
    print('Закрыл сверху')
#     print(result)
#     print(result2)

In [9]:
def request_close_down_pos():
    result = mt5.order_send(request_pep_sell_close) 
    result2 = mt5.order_send(request_ko_buy_close)
    print('Закрыл снизу')

# Бесконечный цикл 1:
## Отрывает позицию в большом объеме и закрывает её как достигнет условие(Условие: тики раз в 1 минуту за последние 20 минут)

In [10]:
#Достаём значения для открытия и закрытия позиции
values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], lotko, lotpep)
Open_Up = values[0]
Open_Down = values[1]
Close = values[2]
#Некоторые переменные для подсчитывания открытых позиции и прибыли
Pos_Up = False
Pos_Down = False
print(Open_Up, Open_Down, Close, Close + (Open_Up - Close) / 2, Close - (Close - Open_Down) / 2)
while(True):
    if datetime.now().minute % 15 == 0 and datetime.now().second == 0:
        if Pos_Up == True:
            Pos_Up = False
            request_close_up_pos()
            print('Закрыл из за того что вышло время')
        if Pos_Down == True:
            Pos_Down = False
            request_close_down_pos()
            print('Закрыл из за того что вышло время')
        values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], lotko, lotpep)
        Open_Up = values[0]
        Open_Down = values[1]
        Close = values[2]
        print(Open_Up, Open_Down, Close, Close + (Open_Up - Close) / 2, Close - (Close - Open_Down) / 2)
        time.sleep(1)
#     last_tick_KO_ask = mt5.symbol_info_tick("KO").ask
#     last_tick_PEP_ask = mt5.symbol_info_tick("PEP").ask
#     last_tick_KO_bid = mt5.symbol_info_tick("KO").bid
#     last_tick_PEP_bid = mt5.symbol_info_tick("PEP").bid
    last_tick_KO_last = mt5.symbol_info_tick("KO").last
    last_tick_PEP_last = mt5.symbol_info_tick("PEP").last

#     last_spread_up = last_tick_PEP_bid - last_tick_KO_ask * lotko / lotpep
#     last_spread_down = last_tick_PEP_ask - last_tick_KO_bid * lotko / lotpep
    last_spread2 = last_tick_PEP_last - last_tick_KO_last * lotko / lotpep
    
    if (last_spread2 <= (Close + (Open_Up - Close) / 2)) and Pos_Up == True:
        Pos_Up = False
        request_close_up_pos()
        print(last_spread2)
    if last_spread2 > Open_Up and Pos_Up == False:
        Pos_Up = True
        request_open_up_pos()
        print(last_spread2)
    if (last_spread2 >= (Close - (Close - Open_Down) / 2)) and Pos_Down == True:    
        Pos_Down = False
        request_close_down_pos()
        print(last_spread2)
    if last_spread2 < Open_Down and Pos_Down == False: 
        Pos_Down = True
        request_open_down_pos()
        print(last_spread2)

4.312000000000012 4.244 4.255999999999972 4.283999999999992 4.249999999999986
Открыл сверху
4.349999999999994
Закрыл сверху
Закрыл из за того что вышло время
4.312000000000012 4.244 4.255999999999972 4.283999999999992 4.249999999999986
Открыл сверху
4.349999999999994
Закрыл сверху
Закрыл из за того что вышло время
4.312000000000012 4.244 4.255999999999972 4.283999999999992 4.249999999999986
Открыл сверху
4.349999999999994


KeyboardInterrupt: 

In [108]:
mt5.shutdown()

True

#  Далее черновой код не использующийся в основной программе